# Manual Content Analysis - Data overview
   
This Jupyter Notebook provides an overview over the articles that were coded for the manual content analysis and the distribution of their labels.

## Load packages

In [1]:
#import relevant packages
import pandas as pd
from pandas import read_excel
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV, ShuffleSplit, GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from pprint import pprint
import spacy
#import German language model
import de_core_news_md
#define nlp pipe
nlp = de_core_news_md.load()

## Read data

In [125]:
#read in the manually coded data
mca = read_excel("mca_data.xlsx")

#read in the article data
#first Dataset A
articles = read_excel("sample.xlsx")
#then Dataset B
articles_online = read_excel("sample_online_new.xlsx")
#then Dataset C
articles2 = read_excel("Dataset C.xlsx")
print(len(articles), len(articles_online), len(articles2))

250 150 98


In [126]:
#add the two print dfs together
articles = articles.append(articles2)
len(articles)

348

In [127]:
articles

,ID,Newspaper,Date,Length,Headline,Article,Author
0,2471,Rheinische Post,2020-05-02,291 words,Die Demontage einer Ministerin,Maximilian Plück Armin Laschet brauchte am Don...,"Plück, Maximilian"
1,6803,Der Tagesspiegel,2020-05-19,418 words,Microsoft mahnt Berlin ab Der US-Konzern verla...,Gegenwind für die Berliner Beauftragte für Dat...,NaN
2,779,Die Welt,2020-04-22,595 words,Bolsonaro verliert die Nerven; Brasiliens Staa...,"Jair Bolsonaro sieht müde aus, mehrfach hustet...",Tobias Käufer
3,3110,Stuttgarter Zeitung,2020-04-30,287 words,Anklage im Mordfall Lübcke,Die Bundesanwaltschaft hat Anklage im Fall des...,AFP
4,3266,Stuttgarter Zeitung,2020-04-25,198 words,Maskenpflicht auchin Schulen?; Bundesbildungsm...,Bundesbildungsministerin Anja Karliczek (CDU) ...,dpa
5,738,Die Welt,2020-04-22,931 words,Einsame Spitze; Das Aus der SAP-Chefin Morgan ...,"Doppelspitzen erfüllen nur selten ihren Zweck,...",Klaus Boldt
6,1122,Die Welt,2020-05-04,872 words,"EU-Solidarität, von allen für alle","Statt gemeinsam die Seuche zu bekämpfen, wurde...",Jürgen Rüttgers
7,5298,Süddeutsche Zeitung (inkl. Regionalausgaben),2020-05-14,611 words,Parkplatz-Kneipen erlaubt; Stadt kommt Wirten ...,"Ein Bier, wo bisher das Auto stand? Schnitzel ...",NaN
8,4761,Süddeutsche Zeitung (inkl. Regionalausgaben),2020-05-08,705 words,Bürgerrechte gelten auch im Notstand; Grüne be...,Landkreis - Die Zuwächse der Grünen sind zulet...,IRIS HILBERTH
9,1538,Rheinische Post,2020-04-23,652 words,CDU fordert Rückbau von zwei Umweltspuren; Tes...,Hendrik Gaasterland Düsseldorf Die CDU-Fraktio...,"Gaasterland, Hendrik"


## Pre-processing

In [128]:
#selecting only the relevant columns
mca = mca[["CID", "AID", "NO", "CPA", "BOV", "BOA", "NEU"]]
#drop the firs row, because it contains the column names
mca = mca.drop(mca.index[0])
#reset the index
mca = mca.reset_index()
#inspect the data
mca.head()

,index,CID,AID,NO,CPA,BOV,BOA,NEU
0,1,03,490,1,3,2,2,2
1,2,03,3414,3,1,1,1,2
2,3,03,6996,4,3,2,2,1
3,4,03,4894,5,3,2,2,1
4,5,03,3110,3,3,2,2,2


In [129]:
#change the datatypes of the manually coded df
mca = mca[mca["AID"].notna()]
mca["AID"] = mca["AID"].astype(int)
mca["CID"] = mca["CID"].astype(int)
mca["NO"] = mca["NO"].astype(int)
mca["CPA"] = mca["CPA"].astype(int)
mca = mca[mca["BOV"].notna()]
mca["BOV"] = mca["BOV"].astype(int)
mca["BOA"] = mca["BOA"].astype(int)
mca["NEU"] = mca["NEU"].astype(int)

In [130]:
articles = articles.rename(columns= {"ID":"AID"})
articles.head()

,AID,Newspaper,Date,Length,Headline,Article,Author
0,2471,Rheinische Post,2020-05-02,291 words,Die Demontage einer Ministerin,Maximilian Plück Armin Laschet brauchte am Don...,"Plück, Maximilian"
1,6803,Der Tagesspiegel,2020-05-19,418 words,Microsoft mahnt Berlin ab Der US-Konzern verla...,Gegenwind für die Berliner Beauftragte für Dat...,NaN
2,779,Die Welt,2020-04-22,595 words,Bolsonaro verliert die Nerven; Brasiliens Staa...,"Jair Bolsonaro sieht müde aus, mehrfach hustet...",Tobias Käufer
3,3110,Stuttgarter Zeitung,2020-04-30,287 words,Anklage im Mordfall Lübcke,Die Bundesanwaltschaft hat Anklage im Fall des...,AFP
4,3266,Stuttgarter Zeitung,2020-04-25,198 words,Maskenpflicht auchin Schulen?; Bundesbildungsm...,Bundesbildungsministerin Anja Karliczek (CDU) ...,dpa


## Merging the dataframes

In [131]:
#change the AID column dtype for the article df -> necessary for merging
articles["AID"] = articles["AID"].astype(int)
#merge the mca data with the article data
df = mca.merge(articles, how="left", on="AID")
#check for duplicates
duplicates = df[df.duplicated(["AID"])]
#check the length
len(df)

575

In [132]:
#rename columns
articles_online = articles_online.rename(columns={"ID":"AID"})
#select only the AID (to merge on) and the text
articles_online = articles_online[["AID", "Article", "Newspaper"]]
#merge the dataframe with the online article data
df = df.merge(articles_online, how="left", on="AID")
#inspect the data
df.head(3)

,index,CID,AID,NO,CPA,BOV,BOA,NEU,Newspaper_x,Date,Length,Headline,Article_x,Author,Article_y,Newspaper_y
0,1,3,490,1,3,2,2,2,Aachener Zeitung,2020-05-15,43 words,FDP lädt ein zur Wahlversammlung,Simmerath Der FDP Ortsverein Simmerath lädt zu...,NaN,NaN,NaN
1,2,3,3414,3,1,1,1,2,Stuttgarter Zeitung,2020-05-15,576 words,Wer stopft das Steuerloch?,Angesichts der gigantischen Steuer-ausfälle im...,Thorsten Knuf,NaN,NaN
2,3,3,6996,4,3,2,2,1,Der Tagesspiegel,2020-05-10,801 words,Schulbetrieb und Klassenfahrt,"""Kindeswohlgefährdung begünstigt. Experten bes...",NaN,NaN,NaN


In [133]:
#fill empty columns
df["Article_x"]= df["Article_x"].fillna("")
df["Article_y"]= df["Article_y"].fillna("") 
#create a unified text column
df["Article"] = df["Article_x"] + df["Article_y"]

#fill empty columns
df["Newspaper_x"]= df["Newspaper_x"].fillna("")
df["Newspaper_y"]= df["Newspaper_y"].fillna("") 
#create a unified text column
df["Newspaper"] = df["Newspaper_x"] + df["Newspaper_y"]

#inspect df
df.head(3)

,index,CID,AID,NO,CPA,BOV,BOA,NEU,Newspaper_x,Date,Length,Headline,Article_x,Author,Article_y,Newspaper_y,Article,Newspaper
0,1,3,490,1,3,2,2,2,Aachener Zeitung,2020-05-15,43 words,FDP lädt ein zur Wahlversammlung,Simmerath Der FDP Ortsverein Simmerath lädt zu...,NaN,,,Simmerath Der FDP Ortsverein Simmerath lädt zu...,Aachener Zeitung
1,2,3,3414,3,1,1,1,2,Stuttgarter Zeitung,2020-05-15,576 words,Wer stopft das Steuerloch?,Angesichts der gigantischen Steuer-ausfälle im...,Thorsten Knuf,,,Angesichts der gigantischen Steuer-ausfälle im...,Stuttgarter Zeitung
2,3,3,6996,4,3,2,2,1,Der Tagesspiegel,2020-05-10,801 words,Schulbetrieb und Klassenfahrt,"""Kindeswohlgefährdung begünstigt. Experten bes...",NaN,,,"""Kindeswohlgefährdung begünstigt. Experten bes...",Der Tagesspiegel


In [134]:
len(df)

575

In [135]:
df["Article"].isnull().sum()

0

## Remove duplicates

In [136]:
#remove duplicates
df.drop_duplicates(subset ="AID", keep = "first", inplace = True) 
len(df)

497

In [137]:
#see if there is missing data
df = df[df["Article"].notna()]
len(df)

497

## Remove codes that don't align with the original data

In [138]:
df = df[df['Newspaper'] != ""]
len(df)

487

## Adapt column values

In [139]:
df["NEU"] = df["NEU"].replace(1, 0)  
df["NEU"] = df["NEU"].replace(2, 1)  #1 = yes
df["BOV"] = df["BOV"].replace(2, 0)  #1 = yes
df["BOA"] = df["BOA"].replace(2, 0)  #1 = yes
#inspect data
df.head(3)

,index,CID,AID,NO,CPA,BOV,BOA,NEU,Newspaper_x,Date,Length,Headline,Article_x,Author,Article_y,Newspaper_y,Article,Newspaper
0,1,3,490,1,3,0,0,1,Aachener Zeitung,2020-05-15,43 words,FDP lädt ein zur Wahlversammlung,Simmerath Der FDP Ortsverein Simmerath lädt zu...,NaN,,,Simmerath Der FDP Ortsverein Simmerath lädt zu...,Aachener Zeitung
1,2,3,3414,3,1,1,1,1,Stuttgarter Zeitung,2020-05-15,576 words,Wer stopft das Steuerloch?,Angesichts der gigantischen Steuer-ausfälle im...,Thorsten Knuf,,,Angesichts der gigantischen Steuer-ausfälle im...,Stuttgarter Zeitung
2,3,3,6996,4,3,0,0,0,Der Tagesspiegel,2020-05-10,801 words,Schulbetrieb und Klassenfahrt,"""Kindeswohlgefährdung begünstigt. Experten bes...",NaN,,,"""Kindeswohlgefährdung begünstigt. Experten bes...",Der Tagesspiegel


In [140]:
df

,index,CID,AID,NO,CPA,BOV,BOA,NEU,Newspaper_x,Date,Length,Headline,Article_x,Author,Article_y,Newspaper_y,Article,Newspaper
0,1,3,490,1,3,0,0,1,Aachener Zeitung,2020-05-15,43 words,FDP lädt ein zur Wahlversammlung,Simmerath Der FDP Ortsverein Simmerath lädt zu...,NaN,,,Simmerath Der FDP Ortsverein Simmerath lädt zu...,Aachener Zeitung
1,2,3,3414,3,1,1,1,1,Stuttgarter Zeitung,2020-05-15,576 words,Wer stopft das Steuerloch?,Angesichts der gigantischen Steuer-ausfälle im...,Thorsten Knuf,,,Angesichts der gigantischen Steuer-ausfälle im...,Stuttgarter Zeitung
2,3,3,6996,4,3,0,0,0,Der Tagesspiegel,2020-05-10,801 words,Schulbetrieb und Klassenfahrt,"""Kindeswohlgefährdung begünstigt. Experten bes...",NaN,,,"""Kindeswohlgefährdung begünstigt. Experten bes...",Der Tagesspiegel
3,4,3,4894,5,3,0,0,0,Süddeutsche Zeitung (inkl. Regionalausgaben),2020-05-12,315 words,CSU UND AFD; Rote Linie überschritten,Hätte Stefan Krimmer die Wahl zum stellvertret...,VON MARTIN MÜHLFENZL,,,Hätte Stefan Krimmer die Wahl zum stellvertret...,Süddeutsche Zeitung (inkl. Regionalausgaben)
4,5,3,3110,3,3,0,0,1,Stuttgarter Zeitung,2020-04-30,287 words,Anklage im Mordfall Lübcke,Die Bundesanwaltschaft hat Anklage im Fall des...,AFP,,,Die Bundesanwaltschaft hat Anklage im Fall des...,Stuttgarter Zeitung
5,6,3,4674,5,1,0,0,1,Süddeutsche Zeitung (inkl. Regionalausgaben),2020-04-22,84 words,Entscheidung über neue Militär-Jets erst später,Berlin - Die Luftwaffe wird wohl noch Jahre au...,MSZ,,,Berlin - Die Luftwaffe wird wohl noch Jahre au...,Süddeutsche Zeitung (inkl. Regionalausgaben)
6,7,3,2478,2,3,0,0,1,Rheinische Post,2020-05-07,85 words,Alexander Schumacher hält Sprechstunde ab,Monheim (elm) Die SPD Monheim steht für alle B...,NaN,,,Monheim (elm) Die SPD Monheim steht für alle B...,Rheinische Post
7,8,3,2264,2,6,0,0,0,Rheinische Post,2020-05-14,152 words,Zu wichtig für Wahlkampf,"Gut drei Jahre ist es her, dass der Siegerentw...","Gruhn, Andreas",,,"Gut drei Jahre ist es her, dass der Siegerentw...",Rheinische Post
8,9,3,5913,5,3,1,1,1,Süddeutsche Zeitung (inkl. Regionalausgaben),2020-05-06,391 words,Im Nachhinein solide; Im umstrittenen Haushalt...,Eching - Weil bei den diversen Echinger Neubau...,KBH,,,Eching - Weil bei den diversen Echinger Neubau...,Süddeutsche Zeitung (inkl. Regionalausgaben)
9,10,3,6483,4,4,1,1,1,Der Tagesspiegel,2020-04-30,673 words,Note: Mangelhaft Der Landeselternausschuss kri...,Der Landeselternausschuss hat den sogenannten ...,Ronja Ringelstein,,,Der Landeselternausschuss hat den sogenannten ...,Der Tagesspiegel


## Inspect data

In [141]:
#initial overview over the number of articles per newspaper
df.groupby("Newspaper").describe()

index                          \
                                              count        mean         std   
Newspaper                                                                     
Aachener Zeitung                               31.0  338.903226  173.931855   
Der Tagesspiegel                               42.0  310.071429  187.818593   
Die Welt                                       28.0  259.714286  167.226805   
Rheinische Post                                77.0  283.896104  175.240480   
Stuttgarter Zeitung                            40.0  297.950000  184.687671   
Süddeutsche Zeitung (inkl. Regionalausgaben)  123.0  299.569106  181.074212   
aachener zeitung (www)                         20.0  349.900000  102.930687   
der tagesspiegel (www)                         39.0  376.743590  104.337438   
die welt (www)                                 25.0  374.080000  109.067609   
rheinische post (www)                          22.0  343.181818  103.791750   
stuttgarter zeitung (www)                      21.0  329.619048  117.793665   
sueddeutschet politik (www)                    19.0  329.473684  116.963607   

                                                                            \
                                                min     25%    50%     75%   
Newspaper                                                                    
Aachener Zeitung                                1.0  202.00  325.0  513.50   
Der Tagesspiegel                                3.0  136.75  343.0  493.25   
Die Welt                                       18.0  126.75  238.5  364.25   
Rheinische Post                                 7.0  133.00  295.0  374.00   
Stuttgarter Zeitung                             2.0  124.75  311.0  496.75   
Süddeutsche Zeitung (inkl. Regionalausgaben)    4.0  114.50  313.0  489.00   
aachener zeitung (www)                        182.0  216.75  387.5  415.00   
der tagesspiegel (www)                        183.0  304.50  410.0  456.50   
die welt (www)                                184.0  226.00  423.0  456.00   
rheinische post (www)                         198.0  218.00  390.0  428.00   
stuttgarter zeitung (www)                     185.0  214.00  404.0  439.00   
sueddeutschet politik (www)                   187.0  197.50  386.0  429.50   

                                                       CID            ...  \
                                                max  count      mean  ...   
Newspaper                                                             ...   
Aachener Zeitung                              576.0   31.0  2.451613  ...   
Der Tagesspiegel                              566.0   42.0  2.690476  ...   
Die Welt                                      551.0   28.0  2.642857  ...   
Rheinische Post                               574.0   77.0  2.389610  ...   
Stuttgarter Zeitung                           572.0   40.0  2.650000  ...   
Süddeutsche Zeitung (inkl. Regionalausgaben)  577.0  123.0  2.536585  ...   
aachener zeitung (www)                        476.0   20.0  1.600000  ...   
der tagesspiegel (www)                        479.0   39.0  1.410256  ...   
die welt (www)                                477.0   25.0  1.400000  ...   
rheinische post (www)                         454.0   22.0  1.636364  ...   
stuttgarter zeitung (www)                     475.0   21.0  1.333333  ...   
sueddeutschet politik (www)                   478.0   19.0  1.368421  ...   

                                              BOA         NEU            \
                                              75%  max  count      mean   
Newspaper                                                                 
Aachener Zeitung                              1.0  1.0   31.0  0.741935   
Der Tagesspiegel                              1.0  1.0   42.0  0.619048   
Die Welt                                      1.0  1.0   28.0  0.392857   
Rheinische Post                               1.0  1.0   77.0  0.701299   
Stuttgar

In [146]:
#label distribution per newspaper

dw = df[df['Newspaper'].isin(["Die Welt", "die welt (www)"])]
print("Die Welt")
print("Balance of actors")
print(dw["BOA"].value_counts())
print("Balance of viewpoints")
print(dw["BOV"].value_counts())
print("Neutrality")
print(dw["NEU"].value_counts())

print("")

sz = df[df['Newspaper'].isin(["Süddeutsche Zeitung (inkl. Regionalausgaben)", "sueddeutschet politik (www)"])]
print("Die Süddeutsche:")
print("Balance of actors")
print(sz["BOA"].value_counts())
print("Balance of viewpoints")
print(sz["BOV"].value_counts())
print("Neutrality")
print(sz["NEU"].value_counts())

print("")

ts = df[df['Newspaper'].isin(["Der Tagesspiegel", "der tagesspiegel (www)"])]   
print("Der Tagesspiegel")
print("Balance of actors")
print(ts["BOA"].value_counts())
print("Balance of viewpoints")
print(ts["BOV"].value_counts())
print("Neutrality")
print(ts["NEU"].value_counts())

print("")

az = df[df['Newspaper'].isin(["Aachener Zeitung", "aachener zeitung (www)"])]
print("Aachener Zeitung")
print("Balance of actors")
print(az["BOA"].value_counts())
print("Balance of viewpoints")
print(az["BOV"].value_counts())
print("Neutrality")
print(az["NEU"].value_counts())

print("")

rp = df[df['Newspaper'].isin(["Rheinische Post", "rheinische post (www)"])]
print("Rheinische Post")
print("Balance of actors")
print(rp["BOA"].value_counts())
print("Balance of viewpoints")
print(rp["BOV"].value_counts())
print("Neutrality")
print(rp["NEU"].value_counts())

print("")

stz = df[df['Newspaper'].isin(["Stuttgarter Zeitung", "stuttgarter zeitung (www)"])]
print("Stuttgarter Zeitung")
print("Balance of actors")
print(stz["BOA"].value_counts())
print("Balance of viewpoints")
print(stz["BOV"].value_counts())
print("Neutrality")
print(stz["NEU"].value_counts())

Die Welt
Balance of actors
0    28
1    25
Name: BOA, dtype: int64
Balance of viewpoints
1    29
0    24
Name: BOV, dtype: int64
Neutrality
0    30
1    23
Name: NEU, dtype: int64

Die Süddeutsche:
Balance of actors
0    93
1    49
Name: BOA, dtype: int64
Balance of viewpoints
0    73
1    69
Name: BOV, dtype: int64
Neutrality
1    74
0    68
Name: NEU, dtype: int64

Der Tagesspiegel
Balance of actors
0    57
1    24
Name: BOA, dtype: int64
Balance of viewpoints
1    41
0    40
Name: BOV, dtype: int64
Neutrality
1    41
0    40
Name: NEU, dtype: int64

Aachener Zeitung
Balance of actors
0    31
1    20
Name: BOA, dtype: int64
Balance of viewpoints
0    35
1    16
Name: BOV, dtype: int64
Neutrality
1    36
0    15
Name: NEU, dtype: int64

Rheinische Post
Balance of actors
0    65
1    34
Name: BOA, dtype: int64
Balance of viewpoints
0    54
1    45
Name: BOV, dtype: int64
Neutrality
1    65
0    34
Name: NEU, dtype: int64

Stuttgarter Zeitung
Balance of actors
0    46
1    15
Name: BOA,

In [147]:
#overall distribution of labels
print("Balance of actors")
print(df["BOA"].value_counts())
print("")
print("Balance of vviews")
print(df["BOV"].value_counts())
print("")
print("Neutrality")
print(df["NEU"].value_counts())

Balance of actors
0    320
1    167
Name: BOA, dtype: int64

Balance of vviews
0    263
1    224
Name: BOV, dtype: int64

Neutrality
1    283
0    204
Name: NEU, dtype: int64


In [148]:
#overview of articles per coder
df["CID"].value_counts()

2    166
3    155
1    128
4     38
Name: CID, dtype: int64

## Saving the datafile

In [149]:
df.to_excel("mca_finaldata.xlsx")